In [2]:
# This line is needed for python 2.7 ; probably not for python 3
from __future__ import print_function

import numpy as np
import pandas as pd
import gc
import warnings

from bayes_opt import BayesianOptimization
from sklearn.cross_validation import cross_val_score, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import log_loss, matthews_corrcoef, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import contextlib

In [3]:
application_train   = pd.read_pickle('application_train.p')
bureau_1            = pd.read_pickle('bureau_1.p')
bureau_2            = pd.read_pickle('bureau_2.p')
bureau_3            = pd.read_pickle('bureau_3.p')
bureau_4            = pd.read_pickle('bureau_4.p')
prev_data           = pd.read_pickle('prev_data.p')
pos_data            = pd.read_pickle("pos_data.p")
ins_data            = pd.read_pickle("ins_data.p")
cc_data             = pd.read_pickle("cc_data.p")
application_test    = pd.read_pickle('application_test.p')

In [4]:
application_train = pd.merge(application_train,bureau_1,on = 'SK_ID_CURR',how='left')
application_train = pd.merge(application_train,bureau_2,on = 'SK_ID_CURR',how='left')
application_train = pd.merge(application_train,bureau_3,on = 'SK_ID_CURR',how='left')
application_train = pd.merge(application_train,bureau_4,on = 'SK_ID_CURR',how='left')
application_train = pd.merge(application_train,prev_data,on = 'SK_ID_CURR',how='left')
application_train = pd.merge(application_train,pos_data,on = 'SK_ID_CURR',how='left')
application_train = pd.merge(application_train,ins_data,on = 'SK_ID_CURR',how='left')
application_train = pd.merge(application_train,cc_data,on = 'SK_ID_CURR',how='left')


application_test = pd.merge(application_test,bureau_1,on = 'SK_ID_CURR',how='left')
application_test = pd.merge(application_test,bureau_2,on = 'SK_ID_CURR',how='left')
application_test = pd.merge(application_test,bureau_3,on = 'SK_ID_CURR',how='left')
application_test = pd.merge(application_test,bureau_4,on = 'SK_ID_CURR',how='left')
application_test = pd.merge(application_test,prev_data,on = 'SK_ID_CURR',how='left')
application_test = pd.merge(application_test,pos_data,on = 'SK_ID_CURR',how='left')
application_test = pd.merge(application_test,ins_data,on = 'SK_ID_CURR',how='left')
application_test = pd.merge(application_test,cc_data,on = 'SK_ID_CURR',how='left')

In [ ]:
y_train = application_train['TARGET']
X_train = application_train.drop(['TARGET','SK_ID_CURR'], axis=1)
dtrain  = xgb.DMatrix(X_train, label = y_train)

In [3]:
# Comment out any parameter you don't want to test
def XGB_CV(
          max_depth,
         # n_estimators,    
          gamma,
          subsample,
          colsample_bytree,
          min_child_weight,
          max_delta_step
         ):

    global AUCbest
    global ITERbest

#
# Define all XGboost parameters
#

    paramt = {
              'booster' : 'gbtree',
              'max_depth' : int(max_depth),
              #'n_estimators' : max(min(n_estimators, 50), 0), 
              'gamma' : gamma,
              'eta' : 0.1,
              'objective' : 'binary:logistic',
              'nthread' : 4,
              'silent' : True,
              'eval_metric': 'logloss',
              'subsample' : max(min(subsample, 1), 0),
              'colsample_bytree' : max(min(colsample_bytree, 1), 0),
              'min_child_weight' : min_child_weight,
              'max_delta_step' : int(max_delta_step),
              'seed' : 1001
              }

    folds = 5
    cv_score = 0

    print("\n Search parameters (%d-fold validation):\n %s" % (folds, paramt), file=log_file )
    log_file.flush()

    xgbc = xgb.cv(
                    paramt,
                    dtrain,
                    num_boost_round = 20000,
                    stratified = True,
                    nfold = folds,
#                    verbose_eval = 10,
                    early_stopping_rounds = 100,
                    metrics = 'auc',
                    show_stdv = True
               )

# This line would have been on top of this section
#    with capture() as result:

# After xgb.cv is done, this section puts its output into log file. Train and validation scores 
# are also extracted in this section. Note the "diff" part in the printout below, which is the 
# difference between the two scores. Large diff values may indicate that a particular set of 
# parameters is overfitting, especially if you check the CV portion of it in the log file and find 
# out that train scores were improving much faster than validation scores.

#    print('', file=log_file)
#    for line in result[1]:
#        print(line, file=log_file)
#    log_file.flush()

    val_score   = xgbc['test-auc-mean'].iloc[-1]
    train_score = xgbc['train-auc-mean'].iloc[-1]
    print(' Stopped after %d iterations with train-auc = %f val-auc = %f ( diff = %f ) train-gini = %f val-gini = %f' % ( len(xgbc), train_score, val_score, (train_score - val_score), (train_score*2-1),
(val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(xgbc)

    return (val_score*2) - 1

In [4]:
# Define the log file. If you repeat this run, new output will be added to it
log_file = open('AUC-5fold-XGB-run-01-v1-full.log', 'a')
AUCbest  = -1.
ITERbest = 0

In [5]:
XGB_BO = BayesianOptimization(XGB_CV, {
                                     'max_depth': (2, 12),
                                    # 'n_estimators' : (50, 400),    
                                     'gamma': (0.001, 10.0),
                                     'subsample': (0.4, 1.0),
                                     'colsample_bytree' :(0.4, 1.0),
                                     'min_child_weight': (0, 20),
                                     'max_delta_step': (0,10)
                                    })

NameError: name 'BayesianOptimization' is not defined

In [2]:
print('-'*130)
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)

----------------------------------------------------------------------------------------------------------------------------------


NameError: name 'log_file' is not defined

In [ ]:
#In my version of sklearn there are many warning thrown out by the GP portion of this code. This is set to prevent them from showing on screen.
#If you have a special relationship with your computer and want to know everything it is saying back, you'd probably want to remove the two "warnings" lines and slide the XGB_BO line all the way left.
#I am doing only 2 initial points, which along with 8 exploratory points above makes it 10 "random" parameter combinations. I'd say that 15-20 is usually adequate. For n_iter 25-50 is usually enough.
#There are several commented out maximize lines that could be worth exploring. The exact combination of parameters determines exploitation vs. exploration. It is tough to know which would work better without actually trying, though in my hands exploitation with "expected improvement" usually works the best. That's what the XGB_BO.maximize line below is specifying.

In [1]:
print('-'*130)
print('Final Results')
print('Maximum XGBOOST value: %f' % XGB_BO.res['max']['max_val'])
print('Best XGBOOST parameters: ', XGB_BO.res['max']['max_params'])
print('-'*130, file=log_file)
print('Final Result:', file=log_file)
print('Maximum XGBOOST value: %f' % XGB_BO.res['max']['max_val'], file=log_file)
print('Best XGBOOST parameters: ', XGB_BO.res['max']['max_params'], file=log_file)
log_file.flush()
log_file.close()

history_df = pd.DataFrame(XGB_BO.res['all']['params'])
history_df2 = pd.DataFrame(XGB_BO.res['all']['values'])
history_df = pd.concat((history_df, history_df2), axis=1)
history_df.rename(columns = { 0 : 'gini'}, inplace=True)
history_df['AUC'] = ( history_df['gini'] + 1 ) / 2
history_df.to_csv('AUC-5fold-XGB-run-01-v1-grid.csv')

----------------------------------------------------------------------------------------------------------------------------------
Final Results


NameError: name 'XGB_BO' is not defined